In [3]:
import matplotlib.pyplot as plt
import numpy as np
import itertools
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import umap
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.metrics import roc_curve, auc
import torch



Plotting Confusion Matrix:

In [2]:
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Visualizing with PCA:

In [ ]:
def plot_pca_images(images,labels, dim=2):
    pca = PCA(n_components=dim)
    data_pca = pca.fit_transform(images)

    if dim == 2:
        plt.scatter(data_pca[:, 0], data_pca[:, 1], c=labels)
        plt.title('PCA (2D)')
    elif dim == 3:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(data_pca[:, 0], data_pca[:, 1], data_pca[:, 2], c=labels)
        ax.set_title('PCA (3D)')

    plt.show()

def plot_pca_images_3d_plotly(images, labels):
    pca = PCA(n_components=3)
    data_pca = pca.fit_transform(images)

    fig = px.scatter_3d(x=data_pca[:, 0], y=data_pca[:, 1], z=data_pca[:, 2], color=labels)
    fig.update_layout(title='PCA (3D)')

    fig.show()

Visualizing with UMAP:

In [ ]:
def plot_umap_images(images, labels, dim=2):
    umap_model = umap.UMAP(n_components=dim)
    data_umap = umap_model.fit_transform(images)

    if dim == 2:
        plt.scatter(data_umap[:, 0], data_umap[:, 1], c=labels)
        plt.title('UMAP (2D)')
    elif dim == 3:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(data_umap[:, 0], data_umap[:, 1], data_umap[:, 2], c=labels)
        ax.set_title('UMAP (3D)')

    plt.show()
    
    
def plot_umap_images_3d_plotly(images, labels):
    umap_model = umap.UMAP(n_components=3)
    data_umap = umap_model.fit_transform(images)

    fig = px.scatter_3d(x=data_umap[:, 0], y=data_umap[:, 1], z=data_umap[:, 2], color=labels)
    fig.update_layout(title='UMAP (3D)')

    fig.show()

Visualizing with t-SNE:

In [ ]:
def plot_tsne_images(images, labels, dim=2):
    tsne = TSNE(n_components=dim)
    images_array = np.array(images)
    data_tsne = tsne.fit_transform(images_array)

    if dim == 2:
        plt.scatter(data_tsne[:, 0], data_tsne[:, 1], c=labels)
        plt.title('t-SNE (2D)')
    elif dim == 3:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(data_tsne[:, 0], data_tsne[:, 1], data_tsne[:, 2], c=labels)
        ax.set_title('t-SNE (3D)')

    plt.show()
    
    
def plot_tsne_images_3d_plotly(images, labels):
    tsne = TSNE(n_components=3)
    images_array = np.array(images)
    data_tsne = tsne.fit_transform(images_array)

    fig = px.scatter_3d(x=data_tsne[:, 0], y=data_tsne[:, 1], z=data_tsne[:, 2], color=labels)
    fig.update_layout(title='t-SNE (3D)')

    fig.show()

Printing AUC Curve:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


from sklearn.preprocessing import label_binarize

def plot_auc_curve_multiclass(model, dataloader, num_classes):
    model.eval()
    y_true = []
    y_pred_probabilities = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            y_pred_probabilities.extend(probabilities.cpu().numpy())
            y_true.extend(targets.cpu().numpy())

    y_true = label_binarize(y_true, classes=range(num_classes))
    y_pred_probabilities = np.array(y_pred_probabilities)

    # Compute ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred_probabilities[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot ROC curves for each class
    plt.figure(figsize=(8, 6))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve (Class {i}) (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve (Multiclass)')
    plt.legend(loc='lower right')
    plt.show()
